In [1]:
import pandas as pd

In [2]:
df=pd.read_csv('train.csv')

In [3]:
df.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [4]:
df=df.dropna()

In [5]:
x=df.drop('label',axis=1)

In [6]:
 y=df['label']
y.head()

0    1
1    0
2    1
3    1
4    1
Name: label, dtype: int64

In [7]:
print(x.shape)
print(y.shape)

(18285, 4)
(18285,)


In [8]:
import tensorflow as tf
tf.__version__

'2.6.0'

In [9]:
from tensorflow.keras.layers import Embedding
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.preprocessing.sequence import pad_sequences


In [10]:
voc_size=5000

In [11]:
messages=x.copy()

In [12]:
messages.reset_index(inplace=True)

In [13]:
import nltk
import re
from nltk.corpus import stopwords

In [14]:
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
import re
ps=PorterStemmer()
corpus=[]
for i in range(0,len(messages)):
    review=re.sub('[^a-zA-Z]',' ',messages['title'][i])
    review=review.lower()
    review=review.split()
    review=[ps.stem(word) for word in review if not word in stopwords.words('english')]
    review=' '.join(review)
    corpus.append(review)

In [15]:
corpus

['hous dem aid even see comey letter jason chaffetz tweet',
 'flynn hillari clinton big woman campu breitbart',
 'truth might get fire',
 'civilian kill singl us airstrik identifi',
 'iranian woman jail fiction unpublish stori woman stone death adulteri',
 'jacki mason hollywood would love trump bomb north korea lack tran bathroom exclus video breitbart',
 'beno hamon win french socialist parti presidenti nomin new york time',
 'back channel plan ukrain russia courtesi trump associ new york time',
 'obama organ action partner soro link indivis disrupt trump agenda',
 'bbc comedi sketch real housew isi caus outrag',
 'russian research discov secret nazi militari base treasur hunter arctic photo',
 'us offici see link trump russia',
 'ye paid govern troll social media blog forum websit',
 'major leagu soccer argentin find home success new york time',
 'well fargo chief abruptli step new york time',
 'anonym donor pay million releas everyon arrest dakota access pipelin',
 'fbi close hilla

In [16]:
onehot_repr=[one_hot(words,voc_size)for words in corpus]
onehot_repr

[[869, 1392, 2081, 2519, 612, 3877, 2655, 1045, 4456, 2379],
 [958, 3725, 4092, 2773, 1110, 262, 1049],
 [3925, 2735, 966, 1685],
 [1591, 841, 4988, 4611, 1646, 2307],
 [1333, 1110, 3538, 3303, 49, 849, 1110, 4235, 1864, 1500],
 [1716,
  3583,
  2792,
  520,
  965,
  3734,
  3324,
  1581,
  658,
  989,
  1856,
  3053,
  4405,
  3633,
  1049],
 [3540, 1970, 4322, 1242, 4830, 876, 1074, 314, 3572, 3652, 1283],
 [3097, 3123, 4868, 1531, 488, 1855, 3734, 1868, 3572, 3652, 1283],
 [2059, 325, 4448, 1, 2118, 3977, 3161, 4747, 3734, 312],
 [639, 908, 1854, 4777, 292, 3569, 3444, 2578],
 [1540, 4299, 2770, 4216, 1943, 2571, 3318, 3110, 1004, 1839, 457],
 [4611, 3172, 612, 3977, 3734, 488],
 [1032, 1693, 2932, 3897, 242, 1912, 4916, 3081, 3401],
 [2889, 4236, 3583, 3390, 4846, 1002, 2974, 3572, 3652, 1283],
 [4436, 711, 413, 442, 4111, 3572, 3652, 1283],
 [3884, 3013, 3082, 3360, 2086, 1373, 1140, 672, 4654, 3699],
 [3530, 1732, 3725],
 [1758, 4579, 436, 1878, 3734, 1001, 3451, 1049],
 [1618, 5

In [17]:
sent_length=20
embed_doc=pad_sequences(onehot_repr,padding='pre',maxlen=sent_length)
embed_doc

array([[   0,    0,    0, ..., 1045, 4456, 2379],
       [   0,    0,    0, ..., 1110,  262, 1049],
       [   0,    0,    0, ..., 2735,  966, 1685],
       ...,
       [   0,    0,    0, ..., 3572, 3652, 1283],
       [   0,    0,    0, ..., 3036, 1522, 1187],
       [   0,    0,    0, ..., 1465,  432,  606]])

In [42]:
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Conv1D
from tensorflow.keras.layers import MaxPooling1D
from tensorflow.keras.layers import Flatten

In [47]:
embedding_vector_features=40
model=Sequential()
model.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length))
model.add(Conv1D(input_dim=sent_length,kernel_size=3, filters=10,padding='valid',activation='relu'))
model.add(MaxPooling1D())
model.add(Conv1D(kernel_size=3, filters=10,padding='valid',activation='relu'))
model.add(MaxPooling1D())
model.add(Flatten())
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam', metrics=['accuracy'])
print(model.summary())

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_3 (Embedding)      (None, 20, 40)            200000    
_________________________________________________________________
conv1d_8 (Conv1D)            (None, 18, 10)            1210      
_________________________________________________________________
max_pooling1d_6 (MaxPooling1 (None, 9, 10)             0         
_________________________________________________________________
conv1d_9 (Conv1D)            (None, 7, 10)             310       
_________________________________________________________________
max_pooling1d_7 (MaxPooling1 (None, 3, 10)             0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 30)                0         
_________________________________________________________________
dense_2 (Dense)              (None, 1)                

In [48]:
import numpy as np
x_final=np.array(embed_doc)
y_final=np.array(y)

In [49]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(x_final,y_final,test_size=0.33, random_state=42)

In [50]:
model.fit(x_train,y_train,validation_data=(x_test,y_test),epochs=100,batch_size=64)

Epoch 1/100
192/192 [==============================] - 1s 5ms/step - loss: 0.5502 - accuracy: 0.6980 - val_loss: 0.3679 - val_accuracy: 0.8295
Epoch 2/100
192/192 [==============================] - 1s 4ms/step - loss: 0.2897 - accuracy: 0.8782 - val_loss: 0.3305 - val_accuracy: 0.8507
Epoch 3/100
192/192 [==============================] - 1s 4ms/step - loss: 0.1984 - accuracy: 0.9260 - val_loss: 0.3499 - val_accuracy: 0.8520
Epoch 4/100
192/192 [==============================] - 1s 4ms/step - loss: 0.1406 - accuracy: 0.9520 - val_loss: 0.3816 - val_accuracy: 0.8490
Epoch 5/100
192/192 [==============================] - 1s 4ms/step - loss: 0.0945 - accuracy: 0.9711 - val_loss: 0.4288 - val_accuracy: 0.8482
Epoch 6/100
192/192 [==============================] - 1s 4ms/step - loss: 0.0612 - accuracy: 0.9844 - val_loss: 0.4882 - val_accuracy: 0.8399
Epoch 7/100
192/192 [==============================] - 1s 4ms/step - loss: 0.0397 - accuracy: 0.9907 - val_loss: 0.5496 - val_accuracy: 0.8401

Epoch 58/100
192/192 [==============================] - 1s 4ms/step - loss: 8.6684e-04 - accuracy: 0.9997 - val_loss: 1.6537 - val_accuracy: 0.8192
Epoch 59/100
192/192 [==============================] - 1s 4ms/step - loss: 8.8920e-04 - accuracy: 0.9997 - val_loss: 1.6706 - val_accuracy: 0.8141
Epoch 60/100
192/192 [==============================] - 1s 4ms/step - loss: 7.1241e-04 - accuracy: 0.9998 - val_loss: 1.6923 - val_accuracy: 0.8210
Epoch 61/100
192/192 [==============================] - 1s 4ms/step - loss: 6.2582e-04 - accuracy: 0.9998 - val_loss: 1.7095 - val_accuracy: 0.8255
Epoch 62/100
192/192 [==============================] - 1s 4ms/step - loss: 5.7357e-04 - accuracy: 0.9998 - val_loss: 1.7449 - val_accuracy: 0.8255
Epoch 63/100
192/192 [==============================] - 1s 4ms/step - loss: 6.8253e-04 - accuracy: 0.9998 - val_loss: 1.7709 - val_accuracy: 0.8220
Epoch 64/100
192/192 [==============================] - 1s 4ms/step - loss: 5.9458e-04 - accuracy: 0.9998 - val_

In [39]:
predict_x=model.predict(x_test) 
pred=np.round(predict_x).astype(int)
print(y_test)
print(pred)
print(y_test.shape)

print(pred.shape)

[1 0 0 ... 0 1 1]
[[[1]
  [1]
  [1]]

 [[1]
  [1]
  [1]]

 [[1]
  [0]
  [0]]

 ...

 [[0]
  [0]
  [1]]

 [[1]
  [1]
  [1]]

 [[1]
  [1]
  [0]]]
(6035,)
(6035, 3, 1)


In [38]:
from sklearn.metrics import confusion_matrix
from sklearn import metrics
print("CNN Data")
pred=array(pred)
print(pred)
score=metrics.accuracy_score(y_test, pred)
print("accuracy: %0.3f" %score)
cm=metrics.confusion_matrix(y_test,pred)
cm

CNN Data


NameError: name 'array' is not defined

In [27]:
from sklearn.metrics import classification_report
print(classification_report(y_test,pred))

ValueError: Classification metrics can't handle a mix of binary and unknown targets

In [28]:
from sklearn.metrics import precision_recall_curve
precision,recall,thres = precision_recall_curve(y_test,pred)
tpr=cm[0][0]/(cm[0][0]+cm[1][0])
fpr=cm[0][1]/(cm[0][1]+cm[1][1])
print("tpr:",tpr)
print("fpr:",fpr)

ValueError: y should be a 1d array, got an array of shape (6035, 3, 1) instead.